In [ ]:
# 장르 속성을 이용한 영화 컨텐츠 기반 필터링

# 데이터 로드 및 확인
- imdb 닷컴에 영화 관련 데이터 조회 가능 (예산, 평점, 시놉시스, 장르 등)
- genres / keywords는 리스트 안의 딕셔너리 객체로 되어있으므로 전처리 후 키워드 값만 남겨두어야한다.

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
movies = pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

In [2]:
# 장르 유사성으로 추천하는 알고리즘

## 분석에 사용할 주요 컬럼 추출
- id, title, genres, vote_average(평균 평점), vote_count(평점 투표수), popularity(영화 인기도), keyword, overview(영화 개요)

In [3]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
movies_df.columns
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [5]:
movies_df[['genres', 'keywords']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4803 non-null   object
 1   keywords  4803 non-null   object
dtypes: object(2)
memory usage: 75.2+ KB


In [6]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [7]:
movies_df['genres'].values[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

## 컬럼 값을 List 객체로 변환하기

### eval()와 literal_eval()의 차이
- 공통점 : 일종의 evaluate 평가 함수
- 차이점 :
    - eval() : 문자형태로 되어있는 표현식을 실행하는 함수 / 문자 형태로 되어있는 데이터를 곧바로 연산 (괄호 해석) 및 함수나 객체 실행하는 강력하고 위험한 함수
    - literal_eval() : 파이썬에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능한 덜 강력한 함수

In [8]:
eval('1+2+(3+3)*2')

15

### 컬럼 값을 문자열 -> List 객체로 변환하기

In [9]:
# 파이썬의 literal_eval() 함수를 통해 'genre'컬럼과 'keywords' 컬럼의 값을 List 객체로 변환하기
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# 장르 컬럼의 값을 하나씩 가져와서 literal_eval에 적용하여 문자열 -> 리스트 객체로 바꾸기

In [10]:
movies_df['genres'].values[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

### 특정 키의 값만 원소로 추출 및 리스트로 생성

In [11]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ v['name'] for v in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ v['name'] for v in x ])
# list comprehension [ ] : 리스트의 형태로 리턴
# x : 리스트 전부 다 # movies_df['genres'].values[0]
# v : 딕셔너리 하나씩 # {'id': 28, 'name': 'Action'}
# v['name'] : name 키의 value

In [12]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


# 장르 콘텐츠 유사도 측정
- 1. 문자열로 변환된 'genres' 컬럼은 카운트 기반으로 피처 벡터화 변환 (사이킷런의 CountVectorizer 라는 클래스 이용)
    - bow : bag of word / 문장을 전부 단어화 시키고 가방에 넣은 후, 가방에서 단어를 하나씩 꺼내서 카운팅 및 유사도 측정
- 2. 'genre' 문자열 -> 피처 백터화 행렬로 변환된 데이터셋을 코사인 유사도를 통해 비교
- 3. 장르 유사도 (코사인 유사도)가 높은 영화 중에 평점이 높은 순으로 정렬하여 영화를 추천

**[참고] CounterVectorizer 사용법**
- 텍스트에서 단위별 출현 횟수를 카운팅하여 수치 벡터화한다.

In [13]:
# 장르별로 일련번호를 매긴다
# [Action, Adventure, Fantasy, Science Fiction]
# 각 영화별로 해당 장르가 몇 번씩 들어가있는 지 카운트

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,1))
# vectorizer = CountVectorizer(ngram_range=(1,2))
# ngram_range :모델의 단어 순서를 보강하기 위한 범위 (범위 최소값, 범위 최대값)
# ex) (1,1) : 단어를 1개씩 피처로 추출 (start=1, end=1) -> 공백으로 끊어 사전 생성
# ex) (1,2) : 토큰화된 (끊어진) 단어를 1개씩, 그리고 순서대로 2개씩 묶어서 피처로 추출 -> 한글에 적합 : '첫번째', '문서', '테스트', '첫번째 문서', '문서 테스트', '두번째', '두번째 문서'
# ex) 자연어처리 : 영어는 공백 단위로 구분하여 자연어 처리가 용이하지만, 한글은 조사 및 띄어쓰기에 따라 의미가 달라지므로 자연어 처리가 까다롭다.

vectorizer.fit(['첫번째 문서 테스트', '두번째 문서 테스트'])
# 4개의 어휘를 학습한 CountVectorizer을 생성
print(vectorizer.vocabulary_)
# 중복되지 않은 고유한 unique 단어에 인덱스 부여

counts = vectorizer.transform(['직접 첫번째 테스트 두번째 테스트'])
# 문자열 목록을 가져와서 미리 학습해놓은 사전을 기반으로 단어의 빈도수를 카운트
print(counts.toarray())
# 학습된 인덱스에 따라 단어 수를 카운트

{'첫번째': 2, '문서': 1, '테스트': 3, '두번째': 0}
[[1 0 1 2]]


In [29]:
movies_df['genres_text'] = movies_df['genres'].apply(lambda x : (' ').join(x))
# x : genres 컬럼이 갖고 있는 리스트 객체가 넘어옴
# (' ').join(x) : 공백을 기준으로 하나의 문장으로 합칠 것
movies_df['genres_text'].head()
# 0    Action Adventure Fantasy Science Fiction


count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# min_df : 전체 문서 내에서 빈도수가 낮은 단어를 제외하기 위한 파라미터
# min_df=0 : 0번 이하로 나타나는 단어는 제외 -> 즉, 모든 단어를 추출하겠다.
# min_df=2 : 전체 문서 내에서 2번 이하로 나타나는 단어는 추출에서 제외
# ngram_range :모델의 단어 순서를 보강하기 위한 범위 (범위 최소값, 범위 최대값)
# ngram_range=(1,1) : 단어를 1개씩 피처로 추출 (start=1, end=1) -> 공백으로 끊어 사전 생성
# ngram_range=(1,2) : 토큰화된 (끊어진) 단어를 1개씩, 그리고 순서대로 2개씩 묶어서 피처로 추출 -> 한글에 적합


genre_mat = count_vect.fit_transform(movies_df['genres_text'])
# fit과 transform을 동시에 : 'genres_text' 컬럼 내 모든 텍스트를 학습하여 사전을 만들어 낸 후, // 첫 번째 영화에 대한 빈도수, 두 번째 영화에 대한 빈도수.. 등 'genres_text' 컬럼 내 각 행의 빈도수를 추출
print(genre_mat.shape) # (4803, 276)
# 4803: 영화의 개수 // 276: CountVectorizer에 의해 생성된 단어 사전의 목록 개수
print(genre_mat.toarray()[:1]) # 276개의 장르 중 9개

(4803, 276)
[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [16]:
# 영화 장르와 영화 장르간의 코싸인 유사도 만들기
# 코싸인 유사도가 높다 = 장르의 유사도가 높다
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
# (4803, 4803) : 대각행렬은 자기자신에 대한 유사도이므로 "1"이고, 대각행렬을 기준으로 좌우대칭되는 행렬이 형성된다
print(genre_sim[:3])
# 첫 & 두 번째 영화에 대한 유사도 : 0.60
# 첫 & 세 번째 영화에 대한 유사도 : 0.45
# 유사도가 클수록, 상호 간의 영화는 '장르'에 있어서 유사도가 높다.

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [17]:
# 첫 번째 영화와 유사도가 높은 n개의 인덱스 값을 뽑아내기
# argsort : 정렬한 후에, 정렬 전의 original 위치값을 찾아내는 함수
genre_sim_sorted_idx = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_idx[:1])
# [:,::-1] : 내림차순
# [:1] : top_n의 idx 조회 : 첫 번째 영화와 장르 유사도가 높은 영화의 movie_df 상에서의 idx 값
# 0 : 자기 자신과 장르 유사도가 가장 높다
# 3494 : 첫 번째 영화와 장르 유사도가 가장 높은 영화는 3494번째 영화
# -> 즉, 자기 자신을 나타내는 0번 레코드를 제외하면 3494번쨰 영화가 장르 기준으로 가장 유사도가 높은 영화이다.

[[   0 3494  813 ... 3038 3037 2401]]


# 장르 콘텐츠 필터링을 이용한 영화 추천 

In [18]:
# movie_df에 idx를 넣으면 어떤 영화인 지 알 수 있음
def fin_sim_movie(df, sorted_ind, title_name, top_n=10):
# df : 영화 정보를 가지고 있는 DataFrame
# sorted_ind : 코싸인 유사도가 높은 순서대로 정렬된 영화의 inx 값
# title_name : 추천 기준이 되는 영화 제목 (해당 영화를 기준으로 장르 유사도가 높은 영화를 추천받게 되는 알고리즘을 만드는 것)
# top_n : 추천 영화수 (유사도가 높은 상위 n개만 추천)
    target_movie = df[df['title']==title_name]
    title_index = target_movie.index.values # index의 value 가져오기
    similar_indexes = sorted_ind[title_index, :(top_n)]
    # sorted_ind 인자로 입력된 genre_sim_sorted_idx 객체에서 유사도가 높은 top_n개의 영화 inx를 추출하는 것
    # 행 : title_index, 열 : top n개까지
    # 해당 영화와 유사도가 높은 top n개 슬라이싱하여 추출하기
    # print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    # reshape, squeeze, flatten 등의 함수를 통해 2차원 -> 1차원으로 차원 축소
    
    return df.iloc[similar_indexes]

In [19]:
sim_movies = fin_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
# 갓파더와 장르상 유사도가 높은 영화 top 10개의 인덱스에 따른 영화 정보
# 2731 : 갓파더 자신
sim_movies[['title', 'vote_average']]

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


# 장르 유사도 & 영화 평점 필터링

- 좀 더 많은 후보군을 선정한 뒤에 **영화의 평점에 따라 필터링**하는 방식으로 변경 필요
    - 설명 : 장르의 유사도로만 상위 10개 영화 추천하지 말고, 평점(낮은건 빼고, 높은 것 위주로 추천. 단, 투표 인원수(신뢰할만한 많은 투표수)도 고려하기)이 높은 상위 10개 영화 추천해주기
    - 방법 : 장르가 유사한 영화에 대해 좀 더 많은 후보군을 만들어둔 후, 평점 높고 투표 인원수도 높은 것을 다시 추려서 10개를 추천해주면 더 의미 있을 것

## 가중평점 컬럼 만들기

- "가중 평점 방식" : 왜곡된 평점 데이터를 피하기 위해 평점에 평가 횟수를 반영한 가중 평점 방식 적용 필요 (IMDB에서 평점을 메기는 방식) 
        - 가중평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
            - v : 개별 영화에 평점을 투표한 횟수
            - m : 평점을 부여하기 위한 최소 투표 횟수 (전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함 <임의값>)
            - R : 개별 영화에 대한 평균 평점
            - C :전체 영화에 대한 평균 평점

In [20]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [21]:
# m : 평점을 부여하기 위한 최소 투표 횟수 (전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함 <임의값>)
# C :전체 영화에 대한 평균 평점

C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 60분위에 해당하는 값 가져옴
print(f'C: {C:.3f}, m: {m:.3f}')

C: 6.092, m: 370.200


In [22]:
# v : 개별 영화에 평점을 투표한 횟수
# R : 개별 영화에 대한 평균 평점

def weighted_vote_average(data):
    v = data['vote_count']
    R = data['vote_average']
    
    return (v/(v+m)) * R + (m/(v+m)) * C

# 가중 평점을 저장할 새로운 컬럼 만들기
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
# axis=1 : 가중평점(weighted_vote_average)의 값들이 행 기준으로 들어갈 것

In [23]:
# weighted_vote를 기준으로 상위 10개 높은 것 조회하기
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


## 장르 & 가중평점

In [24]:
# 기존의 fin_sim_movie 함수 수정하기

def fin_sim_movie(df, sorted_ind, title_name, top_n=10):
# df : 영화 정보를 가지고 있는 DataFrame
# sorted_ind : 코싸인 유사도가 높은 순서대로 정렬된 영화의 inx 값
# title_name : 추천 기준이 되는 영화 제목 (해당 영화를 기준으로 장르 유사도가 높은 영화를 추천받게 되는 알고리즘을 만드는 것)
# top_n : 추천 영화수 (유사도가 높은 상위 n개만 추천)
    target_movie = df[df['title']==title_name]
    title_index = target_movie.index.values # index의 value 가져오기
    
    ### top_n의 2배에 해당하는 장르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    # sorted_ind 인자로 입력된 genre_sim_sorted_idx 객체에서 유사도가 높은 top_n개의 영화 inx를 추출하는 것
    # 행 : title_index, 열 : top n*2개까지 : "후보군을 늘리기 위해 2배 추출"
    # 해당 영화와 유사도가 높은 top n*2개 슬라이싱하여 추출하기
    # print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    # reshape, squeeze, flatten 등의 함수를 통해 2차원 -> 1차원으로 차원 축소
    
    ### 기준이 되는 영화 (본인)은 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # movie_df에 idx를 넣으면 어떤 영화인 지 알 수 있음
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n] # 가중평점이 높은 순서대로 정렬하여 top n개 추출

In [25]:
sim_movies = fin_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
# 갓파더와 장르상 유사도가 높은 영화 중 가중평균가 높은 top 10개의 인덱스에 따른 영화 정보
sim_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
